In [ ]:
# Neural Machine Translation using LSTM for English to Marathi( it can be trained for any language)


In [1]:
# mount the google drive
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
#path for the files
py_file_location = "/content/gdrive/My Drive/Colab Notebooks/"

In [4]:
# download the data from http://www.manythings.org/anki/


In [5]:
# import the libraries
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

In [6]:
import pandas as pd
lines=pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/mar.txt',sep="\t",header=None)
lines.head()

,0,1,2
0,Go.,जा.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Run!,पळ!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
2,Run!,धाव!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run!,पळा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,धावा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [7]:
lines.shape

(40751, 3)

In [8]:
## Renamng the columns
lines.columns=['eng','mar','france']
lines.head()

,eng,mar,france
0,Go.,जा.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Run!,पळ!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
2,Run!,धाव!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run!,पळा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,धावा!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [9]:
# drop the france column
col_to_drop=['france']
lines=lines.drop(col_to_drop,axis=1)
lines.head()

,eng,mar
0,Go.,जा.
1,Run!,पळ!
2,Run!,धाव!
3,Run!,पळा!
4,Run!,धावा!


In [10]:
# lowercase all the characters
# 
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.mar=lines.mar.apply(lambda x: x.lower())

In [11]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.mar=lines.mar.apply(lambda x: re.sub("'", '', x))

In [12]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.mar=lines.mar.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [13]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.mar = lines.mar.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [14]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.mar=lines.mar.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.mar=lines.mar.apply(lambda x: re.sub(" +", " ", x))

In [15]:
# Add start and end tokens to target sequences
lines.mar = lines.mar.apply(lambda x : 'START_ '+ x + ' _END')

In [16]:
lines.sample(10)

,eng,mar
23450,im going to talk to them,START_ मी त्यांच्याशी बोलणार आहे _END
26562,i cant do anything for you,START_ मी तुमच्यासाठी काहीही करू शकत नाही _END
21376,i already feel different,START_ मला आधीच वेगळं वाटतंय _END
37675,i cant do it alone you have to help me,START_ मी ते एकट्याने करू शकत नाही तुला माझी म...
26672,i never watch documentaries,START_ मी कधीही डॉक्युमेन्टरी बघत नाही _END
36357,some parts of this city are very ugly,START_ या शहराचे काही भाग अतिशय कुरूप आहेत _END
33251,dont ask me such a hard question,START_ मला इतका कठीण प्रश्न विचारू नका _END
24038,tom is going to come back,START_ टॉम परत येणार आहे _END
16996,when will tom do that,START_ टॉम तसं केव्हा करेल _END
29240,french isnt an easy language,START_ फ्रेंच सोपी भाषा नाही _END


In [17]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Marathi 
all_marathi_words=set()
for mar in lines.mar:
    for word in mar.split():
        if word not in all_marathi_words:
            all_marathi_words.add(word)

In [18]:
# Max Length of source sequence
lenght_list=[]
for l in lines.eng:
    lenght_list.append(len(l.split(' ')))

print(lenght_list)
max_length_src = np.max(lenght_list)
max_length_src

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 3, 3, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 3, 

34

In [19]:
# Max Length of target sequence
lenght_list=[]
for l in lines.mar:
    lenght_list.append(len(l.split(' ')))
max_length_tar = np.max(lenght_list)
max_length_tar

37

In [20]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_marathi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_marathi_words)
num_encoder_tokens, num_decoder_tokens

(5686, 13691)

In [44]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens



13694

In [22]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])
target_token_index

{'10': 1,
 '100': 2,
 '300': 3,
 '3ds': 4,
 'START_': 5,
 '_END': 6,
 'a': 7,
 'b': 8,
 'beispiel': 9,
 'h₂o': 10,
 'ntt': 11,
 'tatoebaorg': 12,
 'uk': 13,
 'अ': 14,
 'अँकरेजमार्गे': 15,
 'अँजिलीस': 16,
 'अँटीव्हायरस': 17,
 'अँड': 18,
 'अँडी': 19,
 'अँडीजमधल्या': 20,
 'अँब्युलन्स': 21,
 'अंक': 22,
 'अंकांनी': 23,
 'अंकारा': 24,
 'अंकारात': 25,
 'अंकारामध्ये': 26,
 'अंगठ्या': 27,
 'अंगाला': 28,
 'अंगोला': 29,
 'अंघोळ': 30,
 'अंठ्यांपेक्षा': 31,
 'अंड': 32,
 'अंडरवुडचा': 33,
 'अंडरवेअर': 34,
 'अंडी': 35,
 'अंडींचा': 36,
 'अंडीसुद्धा': 37,
 'अंड्यांवर': 38,
 'अंत': 39,
 'अंतर': 40,
 'अंतराळयात्री': 41,
 'अंतराळयान': 42,
 'अंतराळयानाचं': 43,
 'अंतराळयानातून': 44,
 'अंतराळवीर': 45,
 'अंतराळात': 46,
 'अंतरित': 47,
 'अंतिम': 48,
 'अंदाज': 49,
 'अंदाजे': 50,
 'अंधार': 51,
 'अंधाराची': 52,
 'अंधारात': 53,
 'अंमलात': 54,
 'अंशांकित': 55,
 'अकरा': 56,
 'अकरावीत': 57,
 'अकल्पनीय': 58,
 'अकाउंट': 59,
 'अकाउंटमध्ये': 60,
 'अकॉर्डियन': 61,
 'अक्कलदाढ': 62,
 'अक्वॅरियममध्ये': 63,
 'अक्षर': 64,
 'अक्ष

In [23]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())
reverse_target_char_index

{1: '10',
 2: '100',
 3: '300',
 4: '3ds',
 5: 'START_',
 6: '_END',
 7: 'a',
 8: 'b',
 9: 'beispiel',
 10: 'h₂o',
 11: 'ntt',
 12: 'tatoebaorg',
 13: 'uk',
 14: 'अ',
 15: 'अँकरेजमार्गे',
 16: 'अँजिलीस',
 17: 'अँटीव्हायरस',
 18: 'अँड',
 19: 'अँडी',
 20: 'अँडीजमधल्या',
 21: 'अँब्युलन्स',
 22: 'अंक',
 23: 'अंकांनी',
 24: 'अंकारा',
 25: 'अंकारात',
 26: 'अंकारामध्ये',
 27: 'अंगठ्या',
 28: 'अंगाला',
 29: 'अंगोला',
 30: 'अंघोळ',
 31: 'अंठ्यांपेक्षा',
 32: 'अंड',
 33: 'अंडरवुडचा',
 34: 'अंडरवेअर',
 35: 'अंडी',
 36: 'अंडींचा',
 37: 'अंडीसुद्धा',
 38: 'अंड्यांवर',
 39: 'अंत',
 40: 'अंतर',
 41: 'अंतराळयात्री',
 42: 'अंतराळयान',
 43: 'अंतराळयानाचं',
 44: 'अंतराळयानातून',
 45: 'अंतराळवीर',
 46: 'अंतराळात',
 47: 'अंतरित',
 48: 'अंतिम',
 49: 'अंदाज',
 50: 'अंदाजे',
 51: 'अंधार',
 52: 'अंधाराची',
 53: 'अंधारात',
 54: 'अंमलात',
 55: 'अंशांकित',
 56: 'अकरा',
 57: 'अकरावीत',
 58: 'अकल्पनीय',
 59: 'अकाउंट',
 60: 'अकाउंटमध्ये',
 61: 'अकॉर्डियन',
 62: 'अक्कलदाढ',
 63: 'अक्वॅरियममध्ये',
 64: 'अक्षर',
 65: '

In [24]:
lines = shuffle(lines)
lines.head(10)

,eng,mar
32839,there are many cultures on earth,START_ पृथ्वीवर भरपूर संस्कृती आहेत _END
359,its ours,START_ आपलं आहे _END
19320,everybody here knows me,START_ इथे मला सगळेच ओळखतात _END
26434,he will be at home tomorrow,START_ तो उद्या घरी असेल _END
33870,tom was in prison for three years,START_ टॉम तीन वर्ष तुरुंगात होता _END
5044,tom trusts mary,START_ टॉमचा मेरीवर विश्वास आहे _END
22127,this is a serious matter,START_ हा गंभीर मुद्दा आहे _END
28126,i know youre writing a book,START_ तू पुस्तक लिहितो आहेस हे मला माहीत आहे ...
22188,tom died on october th,START_ टॉम ऑक्टोबरला मेला _END
5496,do you live here,START_ तू इथे राहतोस का _END


In [25]:
# Train - Test Split
X, y = lines.eng, lines.mar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)
X_train.shape, X_test.shape

((36675,), (4076,))

In [26]:
X_train.to_pickle('/content/gdrive/My Drive/Colab Notebooks/X_train.pkl')
X_test.to_pickle('/content/gdrive/My Drive/Colab Notebooks/X_test.pkl')

In [27]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

In [28]:
# Encoder decoder Model architecture
latent_dim = 50

In [29]:
encoder_inputs = Input(shape=(None,))
encoder_inputs

<KerasTensor: shape=(None, None) dtype=float32 (created by layer 'input_1')>

In [45]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_decoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [46]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [47]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

In [48]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 50
#tokenizer = Tokenizer(num_words=20000)

In [49]:
from keras.preprocessing.text import Tokenizer
#tokenizer = Tokenizer(num_words=50000)

In [50]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples/batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples/batch_size)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/50
286/286 [==============================] - 68s 217ms/step - loss: 1.1107 - acc: 0.1712 - val_loss: 0.8856 - val_acc: 0.1872
Epoch 2/50
286/286 [==============================] - 59s 207ms/step - loss: 0.8673 - acc: 0.1998 - val_loss: 0.8308 - val_acc: 0.2284
Epoch 3/50
286/286 [==============================] - 59s 207ms/step - loss: 0.8135 - acc: 0.2352 - val_loss: 0.7968 - val_acc: 0.2479
Epoch 4/50
286/286 [==============================] - 60s 208ms/step - loss: 0.7783 - acc: 0.2566 - val_loss: 0.7685 - val_acc: 0.2663
Epoch 5/50
286/286 [==============================] - 59s 207ms/step - loss: 0.7482 - acc: 0.2736 - val_loss: 0.7449 - val_acc: 0.2860
Epoch 6/50
286/286 [==============================] - 60s 208ms/step - loss: 0.7220 - acc: 0.2941 - val_loss: 0.7245 - val_acc: 0.3030
Epoch 7/50
286/286 [==============================] - 59s 207ms/step - loss: 0.6987 - acc: 0.3133 - val_loss: 0.7063 - val_acc: 0.3204
Epoch 8/50
286/286 [==============================] - 5

In [38]:
num_encoder_tokens

5686

In [51]:
# save the weights
model.save_weights('/content/gdrive/My Drive/Colab Notebooks/nmt_weights.h5')

In [52]:
# load the weights

model.load_weights('/content/gdrive/My Drive/Colab Notebooks/nmt_weights.h5')

In [53]:
# inference model setup and implementation
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [54]:
# Decode the sample sequences
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [55]:
# evaluvation on training dataset
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [56]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: he ran meters in seconds flat
Actual Marathi Translation:  तो बरोब्बर सेकंदात मीटर धावला 
Predicted Marathi Translation:  तो म्हणाला एक माणूस शकतो 


In [57]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: its against my religion
Actual Marathi Translation:  हे माझ्या धर्माविरुद्ध आहे 
Predicted Marathi Translation:  ते माझ्या तास 


In [58]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i have a friend in england
Actual Marathi Translation:  माझा इंग्लंडमध्ये एक मित्र आहे 
Predicted Marathi Translation:  माझा एक मित्र आहे 


In [59]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Marathi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: do you want an example
Actual Marathi Translation:  तुम्हाला उदाहरण हवं आहे का 
Predicted Marathi Translation:  तुम्हाला हवं आहे का 


In [60]:
# Evaluation on Validation Dataset

In [61]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [62]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i want to study abroad
Actual Marathi Translation:  मला परदेशी अभ्यास करायचाय 
Predicted Marathi Translation:  मला अभ्यास करायला हवा आहे 


In [63]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i have sinned
Actual Marathi Translation:  मी पाप केले आहेत 
Predicted Marathi Translation:  मी बघितलं 


In [64]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i want to sing a song
Actual Marathi Translation:  मला गाणं गायचंय 
Predicted Marathi Translation:  मला एक गाणं गायला हवी आहे 


In [65]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: im feeling sick
Actual Marathi Translation:  मला कसंतरीच वाटतंय 
Predicted Marathi Translation:  मला आजारी वाटत आहे 


In [66]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Marathi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Marathi Translation:', decoded_sentence[:-4])

Input English sentence: i was a doctor
Actual Marathi Translation:  मी डॉक्टर होते 
Predicted Marathi Translation:  मी डॉक्टर होतो 


In [67]:
# reference https://github.com/hlamba28/Word-Level-Eng-Mar-NMT/blob/master/WordLevelEngMarNMT.ipynb